In [1]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

%matplotlib inline
import matplotlib.pyplot as plt

from IPython import display

from queue import Queue 
import threading
import gc

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
# from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

torch.set_default_dtype(torch.float32)

DEVICE = torch.device("cpu")

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[WinError 2] 系統找不到指定的檔案。 return_code=None stdout="None" stderr="None" timeout_happened=False>

In [2]:
import gym
env = gym.make('LunarLander-v2')

ModuleNotFoundError: No module named 'gym'

In [4]:
class Actor(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 4)

    def forward(self, state):
        hid = torch.tanh(self.fc1(state))
        hid = torch.tanh(self.fc2(hid))
        return F.softmax(self.fc3(hid), dim=-1)

NameError: name 'nn' is not defined

In [5]:
class ActorAgent():

    def __init__(self, network):
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=0.001)

    def learn(self, log_probs, rewards):
        loss = (-log_probs * rewards).sum()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def sample(self, state):
        action_prob = self.network(torch.FloatTensor(state).to(DEVICE))
        action_dist = Categorical(action_prob)
        action = action_dist.sample()
        log_prob = action_dist.log_prob(action)
        return action.item(), log_prob

In [8]:
actor = Actor().to(DEVICE)
actor_agent = ActorAgent(actor)

NameError: name 'Actor' is not defined

In [7]:
class Critic(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, state):
        hid = torch.ReLU(self.fc1(state))
        hid = torch.ReLU(self.fc2(hid))
        return self.fc3(hid)

NameError: name 'nn' is not defined

In [ ]:
class CriticAgent():
    def __init__(self, network):
        self.network = network
        self.optimizer = optim.SGD(self.network.parameters(), lr=0.001)

    def learn(self, advantage):
        loss = advantage ** 2

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def predict(self, state):
        return self.network(state)

In [6]:
critic = Critic().to(DEVICE)
critic_agent = CriticAgent(critic)

NameError: name 'PolicyGradientNetwork' is not defined

In [ ]:
EPISODE_PER_BATCH = 64 # 每蒐集 64 個 episodes 更新一次 agent
NUM_BATCH = 300        # 總共更新 500 次
GAMMA     = 0.9        # 衰敗率

In [9]:
# 訓練前先確認network處於train的狀態
actor_agent.network.train()
critic_agent.network.train()

avg_total_rewards, avg_final_rewards = [], []

prg_bar = tqdm(range(NUM_BATCH))
prg_bar.set_description(f"Total: ?, Final: ?")

for batch in prg_bar:

    log_probs, rewards = [], []
    total_rewards, final_rewards = [], []
    
    # 蒐集訓練資料
    def f(result_queue):        
        env = gym.make('LunarLander-v2')
        state = env.reset()

        local_log_probs, local_rewards = [], []
        
        while True:

            action, log_prob = actor_agent.sample(state)
            next_state, reward, done, _ = env.step(action)

            local_log_probs.append(log_prob)
            state = next_state
            local_rewards.append(reward)

            if done:
                result_queue.put((local_rewards, local_log_probs))
                del env
                break
    
    result_queue = Queue()
    threads = []
    for i in range(EPISODE_PER_BATCH):
        threads.append(threading.Thread(target=f, args=(result_queue,)))
        threads[-1].start()

    for t in threads:
        t.join()
        local_rewards, local_log_probs = result_queue.get()
        
        final_rewards.append(local_rewards[-1])
        for i in range(len(local_rewards)):
            rate = 1
            for j in local_rewards[i:]:
                local_rewards[i] += j * rate
                rate *= GAMMA
        total_rewards.append(sum(local_rewards))
        rewards += local_rewards
        log_probs += local_log_probs
            

    # 紀錄訓練過程
    avg_total_reward = sum(total_rewards) / len(total_rewards)
    avg_final_reward = sum(final_rewards) / len(final_rewards)
    avg_total_rewards.append(avg_total_reward)
    avg_final_rewards.append(avg_final_reward)
    prg_bar.set_description(f"Total: {avg_total_reward: 4.1f}, Final: {avg_final_reward: 4.1f}")

    # 更新網路
    rewards = np.array(rewards, dtype=np.float32)
    rewards = (rewards - np.mean(rewards)) / (np.std(rewards) + 1e-9)  # 將 reward 正規標準化
    actor_agent.learn(torch.stack(log_probs).to(DEVICE), torch.from_numpy(rewards).to(DEVICE))
    
    del log_probs, rewards, total_rewards, final_rewards, threads
    
    gc.collect()

NameError: name 'actor_agent' is not defined

In [10]:
plt.plot(avg_total_rewards)
plt.title("Total Rewards")
plt.show()

NameError: name 'plt' is not defined

In [11]:
plt.plot(avg_final_rewards)
plt.title("Final Rewards")
plt.show()

NameError: name 'plt' is not defined

In [12]:
actor_agent.network.eval()  # 測試前先將 network 切換為 evaluation 模式

state = env.reset()

img = plt.imshow(env.render(mode='rgb_array'))

total_reward = 0

done = False
while not done:
    action, _ = actro_agent.sample(state)
    state, reward, done, _ = env.step(action)

    total_reward += reward

    img.set_data(env.render(mode='rgb_array'))
    display.display(plt.gcf())
    display.clear_output(wait=True)

NameError: name 'actor_agent' is not defined

In [ ]:
print(total_reward)